<a href="https://colab.research.google.com/github/Ochodus/DBLab_onehot/blob/main/one_hot_encoding(Variant_models)_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn import svm
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn import tree
from sklearn.model_selection import train_test_split
import random
import sys
import joblib

In [4]:
data_subset = pd.read_excel('drive/MyDrive/DBLab_Onehot/subset_data.xlsx') #확장 데이터
data_original = pd.read_excel('drive/MyDrive/DBLab_Onehot/mental_health_medical_data.xlsx') #기존 데이터

In [5]:
data = pd.concat([data_subset, data_original]) #stratify 오류 방지를 위해 각 라벨 별로 최소 2개 이상의 데이터를 가지기 위한 concat

In [6]:
# 증상의 문자열 벗기기 함수
def listing(x):
    x = eval(x)
    return x

#모델 예측 정확도를 구하기 위한 간단한 함수
def getAccuracy(prediction, target):
  count = 0
  for x, y in zip(prediction, target):
    if x == y:
      count += 1  
  return count/len(prediction) * 100

def get_clf_eval(y_test, pred):
  accuracy = accuracy_score(y_test, pred)
  f1 = f1_score(y_test, pred, average='weighted')
  return (accuracy, f1)

In [7]:
data['symptoms'] = data['symptoms'].apply(listing)

In [8]:
#레이블 인코딩(타겟 문자열 불가)
from sklearn.preprocessing import LabelEncoder
items = data['disease']
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)

print('인코딩 변환값:',labels)

인코딩 변환값: [ 0  2  2 ... 11 58  4]


In [9]:
data

,Unnamed: 0,disease,symptoms,department
0,0.0,강박 장애,"[불안, 반복적인 동작, 의심]",정신건강의학과
1,1.0,경계성 인격장애,"[의심, 약물남용, 대인관계 어려움, 조절할 수 없는 충동감, 자살의 위험, 우울,...",정신건강의학과
2,2.0,경계성 인격장애,"[의심, 약물남용, 대인관계 어려움, 조절할 수 없는 충동감, 자살의 위험, 우울]",정신건강의학과
3,3.0,경계성 인격장애,"[약물남용, 대인관계 어려움, 조절할 수 없는 충동감, 자살의 위험, 우울]",정신건강의학과
4,4.0,경계성 인격장애,"[의심, 대인관계 어려움, 조절할 수 없는 충동감, 자살의 위험, 우울]",정신건강의학과
...,...,...,...,...
59,59.0,혈관성 치매,"[언어장애, 인지장애, 감각 이상, 마비]",정신건강의학과
60,60.0,히스테리성 인격장애,[],정신건강의학과
61,61.0,레트 증후군,"[상동적 행동, 인지장애, 운동장애, 정신 지체]",정신건강의학과
62,62.0,폐소공포증,"[불안, 좁은 공간에서의 불안감]",정신건강의학과


In [10]:
X_train = np.array([x for x in data['symptoms']])
Y_train = np.array(labels)

In [11]:
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, shuffle=True, stratify = Y_train, random_state=34) #0.8의 학습데이터, 0.2의 테스트데이터로 분리

In [12]:
x_test

array([list(['오심', '생리불순', '손떨림', '가슴 답답', '가슴 두근거림', '얼굴이 화끈거림', '구강건조', '구토', '두통', '어지러움']),
       list(['오심', '생리불순', '손떨림', '가슴 답답', '가슴 두근거림', '구강건조', '근육통', '권태감', '구토', '복부팽만감', '어지러움']),
       list(['가슴 답답', '가슴 두근거림', '얼굴이 화끈거림', '근육통', '구토', '가슴 쓰림', '두통']),
       ...,
       list(['온몸이 떨림', '오심', '손떨림', '가슴 답답', '가슴 두근거림', '얼굴이 화끈거림', '근육통', '권태감', '구토', '두통']),
       list(['손떨림', '고혈압', '서맥', '청색증', '불안', '기운없음', '불면증', '환각', '환청']),
       list(['구강건조', '권태감', '가슴 쓰림', '어지러움', '저림'])], dtype=object)

# **모델링**

In [13]:
#파라미터 설정
vocab_size = 178
embedding_dim = 8 # 임베딩 차원
max_length = 17      # 증상 최대 개수 
padding_type='post'

In [14]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size)#, oov_token=oov_tok)
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index

In [15]:
word_index

{'손떨림': 1,
 '가슴 두근거림': 2,
 '얼굴이 화끈거림': 3,
 '가슴 답답': 4,
 '오심': 5,
 '두통': 6,
 '어지러움': 7,
 '온몸이 떨림': 8,
 '구토': 9,
 '복부팽만감': 10,
 '구강건조': 11,
 '근육통': 12,
 '권태감': 13,
 '저림': 14,
 '가슴 쓰림': 15,
 '생리불순': 16,
 '불면증': 17,
 '환각': 18,
 '불안': 19,
 '환청': 20,
 '발한': 21,
 '복시': 22,
 '저혈압': 23,
 '오한': 24,
 '기운없음': 25,
 '고혈압': 26,
 '청색증': 27,
 '서맥': 28,
 '숨막히는 느낌': 29,
 '가슴 통증': 30,
 '죽음에 대한 공포': 31,
 '감정 변화': 32,
 '언어장애': 33,
 '초조함': 34,
 '섬망': 35,
 '혼돈': 36,
 '환시': 37,
 '수면장애': 38,
 '감각 이상': 39,
 '기억장애': 40,
 '근육긴장': 41,
 '피로감': 42,
 '실행증': 43,
 '시력 감소': 44,
 '실인증': 45,
 '목의 이물감': 46,
 '빈뇨': 47,
 '빈호흡': 48,
 '판단력장애': 49,
 '복부 불편감': 50,
 '방향감각 상실': 51,
 '조급증': 52,
 '낮은 자존감': 53,
 '의식 변화': 54,
 '대인관계 어려움': 55,
 '우울': 56,
 '건망증': 57,
 '산만함': 58,
 '인지장애': 59,
 '부정맥': 60,
 '대화에 불쑥 끼어듬': 61,
 '망상': 62,
 '호흡곤란': 63,
 '복부 통증': 64,
 '수면 시작이 어려움': 65,
 '수면 유지가 어려움': 66,
 '수면 마비': 67,
 '입수면기의 환각': 68,
 '수면 주기의 변화': 69,
 '인내심이 부족함': 70,
 '수면 중 이상한 행동': 71,
 '낮 시간대의 졸음': 72,
 '수면 무호흡': 73,
 '흉부압박감': 74,
 '와해된 행동':

# ***테스트 데이터셋 변형***

In [16]:
#테스트 데이터셋에 기존 증상을 제외한 일치하지 않는 증상 랜덤으로 추가
def add_offset(y_test, x_test):
  for disease, symptoms in zip(y_test, x_test):
    disease_name =  encoder.classes_[disease]
    ind = -1
    for index, i in enumerate(data_original['disease']):
      if disease_name == i:
        ind = index
    flag = True
    while flag:
      rand_ind = random.randint(1, 177)
      for k, symp in enumerate(word_index):
        if k+1 == rand_ind:
          if symp in data_original.iloc[ind]['symptoms']:
            break
          else:
            symptoms.append(symp)
            flag = False
            break

In [17]:
add_offset(y_test, x_test)
x_test

array([list(['오심', '생리불순', '손떨림', '가슴 답답', '가슴 두근거림', '얼굴이 화끈거림', '구강건조', '구토', '두통', '어지러움', '정체감 혼란 및 변형']),
       list(['오심', '생리불순', '손떨림', '가슴 답답', '가슴 두근거림', '구강건조', '근육통', '권태감', '구토', '복부팽만감', '어지러움', '높아진 자존심']),
       list(['가슴 답답', '가슴 두근거림', '얼굴이 화끈거림', '근육통', '구토', '가슴 쓰림', '두통', '충동 조절 어려움']),
       ...,
       list(['온몸이 떨림', '오심', '손떨림', '가슴 답답', '가슴 두근거림', '얼굴이 화끈거림', '근육통', '권태감', '구토', '두통', '피학증']),
       list(['손떨림', '고혈압', '서맥', '청색증', '불안', '기운없음', '불면증', '환각', '환청', '지남력 장애']),
       list(['구강건조', '권태감', '가슴 쓰림', '어지러움', '저림', '부정맥'])], dtype=object)

In [18]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(x_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(x_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

In [19]:
test_padded

array([[ 5, 16,  1, ...,  0,  0,  0],
       [ 5, 16,  1, ...,  0,  0,  0],
       [ 4,  2,  3, ...,  0,  0,  0],
       ...,
       [ 8,  5,  1, ...,  0,  0,  0],
       [ 1, 26, 28, ...,  0,  0,  0],
       [11, 13, 15, ...,  0,  0,  0]], dtype=int32)

# ***NLP 모델***

In [20]:
Models = [0,0,0,0,0]

In [21]:
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(64, activation='softmax')
])

In [22]:
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 8)             1424      
                                                                 
 global_average_pooling1d (G  (None, 8)                0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 24)                216       
                                                                 
 dense_1 (Dense)             (None, 64)                1600      
                                                                 
Total params: 3,240
Trainable params: 3,240
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
# fit model
num_epochs = 20
#history = model.fit(train_padded, y_train, 
                    #epochs=1000, verbose=2)
history = model.fit(train_padded, y_train, 
                     epochs=num_epochs, verbose=2, 
                     validation_split=0.2)

Epoch 1/20
1539/1539 - 2s - loss: 0.6675 - accuracy: 0.8860 - val_loss: 0.1545 - val_accuracy: 0.9672 - 2s/epoch - 1ms/step
Epoch 2/20
1539/1539 - 2s - loss: 0.0888 - accuracy: 0.9798 - val_loss: 0.0595 - val_accuracy: 0.9867 - 2s/epoch - 1ms/step
Epoch 3/20
1539/1539 - 2s - loss: 0.0438 - accuracy: 0.9897 - val_loss: 0.0403 - val_accuracy: 0.9907 - 2s/epoch - 1ms/step
Epoch 4/20
1539/1539 - 2s - loss: 0.0296 - accuracy: 0.9931 - val_loss: 0.0298 - val_accuracy: 0.9934 - 2s/epoch - 1ms/step
Epoch 5/20
1539/1539 - 2s - loss: 0.0212 - accuracy: 0.9948 - val_loss: 0.0238 - val_accuracy: 0.9955 - 2s/epoch - 1ms/step
Epoch 6/20
1539/1539 - 2s - loss: 0.0159 - accuracy: 0.9959 - val_loss: 0.0193 - val_accuracy: 0.9967 - 2s/epoch - 1ms/step
Epoch 7/20
1539/1539 - 2s - loss: 0.0123 - accuracy: 0.9969 - val_loss: 0.0171 - val_accuracy: 0.9969 - 2s/epoch - 1ms/step
Epoch 8/20
1539/1539 - 2s - loss: 0.0101 - accuracy: 0.9974 - val_loss: 0.0152 - val_accuracy: 0.9970 - 2s/epoch - 1ms/step
Epoch 9/

In [24]:
# 학습된 모델로 테스트 데이터 예측
y_prob = model.predict(test_padded, verbose=2) 
predicted = y_prob.argmax(axis=-1)

encoder.classes_[predicted]

481/481 - 0s - 335ms/epoch - 696us/step


array(['신체형 장애', '신체형 장애', '신체형 장애', ..., '신체형 장애', '금단 현상', '신체형 장애'],
      dtype=object)

In [25]:
acc_mlp, fl_mlp = get_clf_eval(y_test, predicted)

print("Accuracy: " + str(acc_mlp) + ", " + "F1 Score: " + str(fl_mlp))

Accuracy: 0.9957762037819221, F1 Score: 0.9956287697014674


In [26]:
Models[0] = model
joblib.dump(model, './drive/MyDrive/DBLab_Onehot/mlp_model.pkl')

['./drive/MyDrive/DBLab_Onehot/mlp_model.pkl']

# ***SVM***


In [27]:
#SupportVector 모델 학습
#SVM = svm.SVC(C=1.0, kernel='rbf', gamma='auto', probability=True)
#SVM.fit(train_padded, y_train)

In [28]:
#predictions_SVM = SVM.predict(test_padded)
#predictions_SVM

#encoder.classes_[predictions_SVM]

In [29]:
#acc_svm, fl_svm = get_clf_eval(y_test, predictions_SVM)

#print("Accuracy: " + str(acc_svm) + ", " + "F1 Score: " + str(fl_svm))

In [30]:
#Models[1] = SVM
#joblib.dump(SVM, './drive/MyDrive/DBLab_Onehot/naive.pkl') 

# ***Naive Bayes***

In [31]:
#Naive bayes 모델 학습
Naive = naive_bayes.MultinomialNB()
Naive.fit(train_padded, y_train)

MultinomialNB()

In [32]:
predictions_NB = Naive.predict(test_padded)

encoder.classes_[predictions_NB]

array(['히스테리성 인격장애', '히스테리성 인격장애', '신체형 장애', ..., '히스테리성 인격장애', '신체형 장애',
       '섬망'], dtype=object)

In [33]:
acc_nb, fl_nb = get_clf_eval(y_test, predictions_NB)

print("Accuracy: " + str(acc_nb) + ", " + "F1 Score: " + str(fl_nb))

Accuracy: 0.5629345636493599, F1 Score: 0.644769121563156


In [34]:
Models[2] = Naive
joblib.dump(Naive, './drive/MyDrive/DBLab_Onehot/naive.pkl') 

['./drive/MyDrive/DBLab_Onehot/naive.pkl']

# ***Decision Tree***

In [35]:
#DecisionTree 모델 학습
DTC = tree.DecisionTreeClassifier(min_samples_leaf=17)
DTC.fit(train_padded, y_train)

DecisionTreeClassifier(min_samples_leaf=17)

In [36]:
predictions_DTC = DTC.predict(test_padded)

encoder.classes_[predictions_DTC]

array(['신체형 장애', '신체형 장애', '신체형 장애', ..., '신체형 장애', '금단 현상', '신체형 장애'],
      dtype=object)

In [37]:
acc_dtc, fl_dtc = get_clf_eval(y_test, predictions_DTC)

print("Accuracy: " + str(acc_dtc) + ", " + "F1 Score: " + str(fl_dtc))

Accuracy: 0.9726427968029112, F1 Score: 0.9741915764096922


In [38]:
Models[3] = DTC
joblib.dump(DTC, './drive/MyDrive/DBLab_Onehot/DTC.pkl')

['./drive/MyDrive/DBLab_Onehot/DTC.pkl']

---
# 여기서부터 추가 



더이상 입력할 증상이 없으면 종료
90%는 넘겨보자

안되면 어떤 증상을 물어보지?

현재 입력한 증상이 있는 질병중 가장 많이 출현하는 증상을 물어보자 예를 들면

"당신은 (해당 증상)을 가졌을 확률이 높습니다."

전제
1. 환자가 거짓된 증상을 입력하지 않는다
2. 오타가 나지 않는다 >> 뛰어쓰기같은건 replace(" ", "")로 처음부터 지우고 나머지는 다시 하라고 하자

In [39]:
def find_mode(pat_word_index):#입력받은 증상 배열
  ##어떤 질병에 포함된 증상인지
  sample_train = []#증상의 index
  torf = True
  for i in range(len(data['symptoms'])):#전체 증상
    for j in range(len(pat_word_index)):#입력받은 증상
      torf = pat_word_index[j] in data['symptoms'][i]
      if(not torf):break
    if(torf): sample_train.append(i)
    ##모든 증상 시리즈를 다 돌면서 한 시리즈에 환자의 증상이 다 들어가 있으면 torf는 계속 true, 끝까지 true면 해당 질병을 append

  ld_symp = []#주어진 증상 배열을 포함하는 질병들의 증상 목록
  for i in data['symptoms'][sample_train]:
    for j in i:
      ld_symp.append(j)

  max = -1
  max_arr = []
  chk_symp = pat_word_index.copy()
  for i in ld_symp:#어떤 증상 i를 확인해볼까?
    count = 0
    if(i not in chk_symp):
      for j in ld_symp:#이 증상이 몇 개 있는지 배열을 돌아볼까?
        if(i == j): count += 1#i 증상이 j로 나오면 1회 추가
      
      if(count > max):#새로운 최빈값
        max_arr = [i]
        max = count

      elif(count == max):#이것도 최빈값
        max_arr.append(i)

    chk_symp.append(i)
  return max_arr

In [106]:
def getTopBatch(y_prob, batch_size):
  sorted = y_prob.argsort(axis=-1)
  topBatch = sorted[0][-batch_size:] 
  return topBatch

def getMiddleBatch(y_prob, batch_size, start_pos):
  sorted = y_prob.argsort(axis=-1)
  middleBatch = sorted[0][-(batch_size+start_pos):-batch_size] 
  return middleBatch

def unionBatch(batch):
  top = []
  all_symps = []

  for i, disease in enumerate(batch):
    target = data_subset['symptoms'][data_subset['disease'].ne(encoder.classes_[disease]).idxmin()]
    
    target = stringToArr(target)
    
    if i == len(batch):
      top = target

    all_symps = target + all_symps

  if '' in all_symps:
    all_symps.remove('')

  return all_symps

def stringToArr(symp_string):
  del_token = "[']"
  for t in del_token:
    symp_string = symp_string.replace(t, "")
  symp_string = symp_string.split(',')

  arr = []

  for t in symp_string:
    if len(t) > 0 and t[0] == " ":
      t = t[1:]
    arr.append(t)
  return arr

def most_frequent(data):
    return max(data, key=data.count)

##증상 입력받을 배열
def getSymptom(key, symps, start_symps, model_use, add_offset, batch_size=10):
  count = 1
  pat_word_index = []
  asked = [] #물어봤던 증상 배열

  #print("\n위 증상중 해당하는 증상을 하나씩 입력해주세요 (없으면 엔터) : " + start_symps)
  pat_symp = start_symps
  pat_word_index.append(pat_symp)
  asked.append(pat_symp)
  pat_word_array = np.array(pd.Series([pat_word_index]))

  ##데이터를 sequence로 변환해주고 padding 해줍니다.
  pat_sequences = tokenizer.texts_to_sequences(pat_word_array.tolist())
  pat_padded = pad_sequences(pat_sequences, padding=padding_type, maxlen=max_length)

  ##확률 계산
  if model_use == 0:
    y_prob = Models[model_use].predict(pat_padded)
  else:
    y_prob = Models[model_use].predict_proba(pat_padded)
  predicted = y_prob.argmax(axis=-1)

  #두번째 이상의 증상 입력
  batch_offset = 0
  while (y_prob[0][predicted[0]] < 0.9) and (pat_symp != '') and (count <= vocab_size):#1. 확률이 90% 미만, 빈칸이 아님, 최대치만큼 안돌음
    predicted_batch = getMiddleBatch(y_prob, batch_size, batch_offset)
    batch_symps = unionBatch(predicted_batch)
    #################

    for t in batch_symps:
      if t in pat_word_index:
        batch_symps.remove(t)

    print("asked: ", end="")
    print(asked)
    print("remain: ", end="")
    print(batch_symps)

    while len(batch_symps) > 0:
      rand_ind = random.randrange(0, len(batch_symps))

      next_symp = batch_symps[rand_ind]
      if next_symp in asked: #물어봤던 증상일 경우 넘어감
        del batch_symps[rand_ind]
        continue

      #테스트를 위한 자동 응답 코드 추가
      if count == 1 and add_offset:
        if next_symp in symps:
          yn = 'n'
        else:
          yn = 'y'
      else:
        if next_symp in symps:
          yn = 'y'
        else:
          yn = 'n'

      #print("\n" + str(count) + ". " + next_symp + " 증상이 있으십니까? (y/n): " + yn)
      count += 1
      

      #############################

      #yn = input("\n" + str(count) + ". " + next_symp + " 증상이 있으십니까? (y/n): ")

    
      asked.append(next_symp)

      if yn == 'y':
        pat_symp = next_symp
        pat_word_index.append(pat_symp)
        break;
      else:
        del batch_symps[rand_ind]
    
    pat_word_array = np.array(pd.Series([pat_word_index]))


    ##데이터를 sequence로 변환해주고 padding 해줍니다.
    pat_sequences = tokenizer.texts_to_sequences(pat_word_array.tolist())
    pat_padded = pad_sequences(pat_sequences, padding=padding_type, maxlen=max_length)

    ##확률 계산
   
    if model_use == 0:
      y_prob = Models[model_use].predict(pat_padded)
    else:
      y_prob = Models[model_use].predict_proba(pat_padded)

    if predicted[0] == y_prob.argmax(axis=-1)[0] and len(batch_symps) == 0:
      
      print("new: ", end="")
      print(unionBatch(getMiddleBatch(y_prob, batch_size, batch_offset)))
      batch_offset += batch_size
    elif predicted[0] != y_prob.argmax(axis=-1)[0]:
      batch_offset = 0

    predicted = y_prob.argmax(axis=-1)

    

  pat_word_index = pat_word_index[:-1]

  print()
  print("예측이 완료되었습니다.")

  #결과 출력

  ##확률이 90%가 넘어서 멈춘 경우
  if (y_prob[0][predicted[0]] > 0.9):
    print("당신의 증상은 "+str(y_prob[0][predicted[0]]*100)+"%의 확률로 "+encoder.classes_[predicted][0]+" 입니다.")

  ##더이상 입력을 안한 경우, 즉 입력을 완료한 경우
  else:
    print("현재 입력해주신 증상으로는 "+str(y_prob[0][predicted[0]]*100)+"%의 확률로 "+encoder.classes_[predicted][0]+" 입니다.")

  print()

  return (encoder.classes_[predicted][0], count)

# 테스트를 위한 자동 응답 코드 추가
def auto_reply(model_use, test_mode, add_offset, batch_size):
  del_token = "[']"
  key = ""
  full = ""
  total_asked_success = 0
  total_asked_fail = 0
  total_success_num = 0
  total_predict = 0
  disease_num = 0
  acc_arr = []
  ask_num_success_arr = []
  ask_num_fail_arr = []
  ask_num_arr = []

  model_name = ['MLP', '', 'NBC', 'DTC']
  test_mode_discription = ['증상이 4개 이상인 질병', '증상이 4개 미만인 질병', '전체 질병']

  if model_use != 1:
    print(file=g)
    print("사용한 모델: " + model_name[model_use], file=g)
    print("실행 대상: " + test_mode_discription[test_mode], file=g)
    print("------------------------------------------------------", file=g)
    for (key, full) in zip(data_original['disease'], data_original['symptoms']):
      success_in_disease = 0
      asked_in_disease_success = 0
      asked_in_disease_fail = 0
      total_symps = 0
      full_arr = []
      for t in del_token:
        full = full.replace(t, "")
      full = full.split(',')

      for t in full:
        if len(t) > 0 and t[0] == " ":
          t = t[1:]
        full_arr.append(t)

      #test_mode가 0이면 4개 이상만, 1이면 4개 미만만, 그외에는 전부 실행
      if len(full_arr) < 4:
        if test_mode == 0:
          continue
      
      if len(full_arr) >= 4:
        if test_mode == 1:
          continue

      print("예측 질병: " + key)
      print("증상 목록: " , end="")
      for (i, s) in enumerate(full_arr):
        if i != len(full_arr) - 1:
          print(s + ", ", end="")
        else:
          print(s)
      print()

      for i in full_arr:
        print("시작 증상: " + i)
        (prediction, try_num) = getSymptom(key, full_arr, i, model_use, add_offset, batch_size)
        if prediction == key:
          total_success_num += 1
          success_in_disease += 1
          total_asked_success += try_num
          asked_in_disease_success += try_num
        else:
          total_asked_fail += try_num
          asked_in_disease_fail += try_num
        total_predict += 1
        total_symps += 1
      print()
      print("---------------------------------------------------------------------------------------------------------")
      print()
      disease_num += 1

      if total_symps > 0:
        acc_arr.append(success_in_disease/total_symps) # 각 질병의 성공 확률
        print("질병: " + key, file=g)
        print("성공률: " + str(round(success_in_disease/total_symps * 100, 2)), file=g)
        if success_in_disease > 0:
          ask_num_success_arr.append(asked_in_disease_success/success_in_disease) # 각 질병의 성공한 경우 평균 질문 횟수
          print("성공 시 질문 횟수 평균: " + str(round(asked_in_disease_success/success_in_disease, 2)), file=g)
        else:
          print("성공 시 질문 횟수 평균: 성공한 적이 없음", file=g)
        if total_symps - success_in_disease > 0:
          ask_num_fail_arr.append(asked_in_disease_fail/(total_symps - success_in_disease)) # 각 질병의 실패한 경우 평균 질문 횟수 
          print("실패 시 질문 횟수 평균: " + str(round(asked_in_disease_fail/(total_symps - success_in_disease), 2)), file=g)
        else:
          print("실패 시 질문 횟수 평균: 실패한 적이 없음", file=g)
        ask_num_arr.append((asked_in_disease_fail + asked_in_disease_success)/total_symps) # 각 질병의 평균 질문 횟수
        print("질문 횟수 평균: " + str(round((asked_in_disease_fail + asked_in_disease_success)/total_symps, 2)), file=g)
        print(file=g)
      

    success_rate = total_success_num/total_predict # 전체 성공률

    if total_success_num > 0:
      avarage_try_success = total_asked_success/total_success_num # 전체 성공한 경우 평균 질문 횟수
    else:
      avarage_try_success = -1
    if total_predict - total_success_num > 0:
      avarage_try_fail = total_asked_fail/(total_predict - total_success_num) # 전체 실패한 경우 평균 질문 횟수
    else:
      avarage_try_fail = -1
    avarage_try = (total_asked_success + total_asked_fail) / total_predict # 전체 평균 질문 횟수
    
    success_rate_by_disease = sum(acc_arr)/len(acc_arr) # 전체 성공률 (질병 별 평균)
    if len(ask_num_success_arr) > 0:
      avarage_try_by_disease_success = sum(ask_num_success_arr)/len(ask_num_success_arr) # 전체 성공한 경우 평균 질문 횟수 (질병 별 평균)
    else:
      avarage_try_by_disease_success = -1
    if len(ask_num_fail_arr) > 0:
      avarage_try_by_disease_fail = sum(ask_num_fail_arr)/len(ask_num_fail_arr) # 전체 실패한 경우 평균 질문 횟수 (질병 별 평균)
    else:
      avarage_try_by_disease_fail = -1
    avarage_try_by_disease = sum(ask_num_arr)/len(ask_num_arr) # 전체 평균 질문 횟수 (질병 별 평균)

    print("사용한 모델: " + model_name[model_use], file=f)
    print("실행 대상: " + test_mode_discription[test_mode], file=f)
    print("-------------질병 별 평균 없이---------------", file=f)
    print("성공률: " + str(round(success_rate * 100, 2)) + "%", file=f)
    if avarage_try_success == -1:
      print("성공한 예측의 평균 질문 횟수: 성공한 적이 없음", file=f)
    else:
      print("성공한 예측의 평균 질문 횟수: " + str(avarage_try_success), file=f)
    if avarage_try_fail == -1:
      print("실패한 예측의 평균 질문 횟수: 실패한 적이 없음", file=f)
    else:
      print("실패한 예측의 평균 질문 횟수: " + str(avarage_try_fail), file=f)
    print("전체 평균 질문 횟수: " + str(avarage_try), file=f)
    print(file=f)
    print("-------------질병 별 평균 적용---------------", file=f)
    print("성공률: " + str(round(success_rate_by_disease * 100, 2)) + "%", file=f)
    if avarage_try_by_disease_success == -1:
      print("성공한 예측의 평균 질문 횟수: 성공한 적이 없음", file=f)
    else:
      print("성공한 예측의 평균 질문 횟수: " + str(avarage_try_by_disease_success), file=f)
    if avarage_try_by_disease_fail == -1:
      print("실패한 예측의 평균 질문 횟수: 실패한 적이 없음", file=f)
    else:
      print("실패한 예측의 평균 질문 횟수: " + str(avarage_try_by_disease_fail), file=f)
    print("전체 평균 질문 횟수: " + str(avarage_try_by_disease), file=f)
    print(file=f)
    print(file=f)

  #################################
f = open('./drive/MyDrive/DBLab_Onehot/result_total.txt', 'w')
g = open('./drive/MyDrive/DBLab_Onehot/result_by_disease.txt', 'w')
for model in range(4):
  for mode in range(3):
    auto_reply(model_use=model, test_mode=mode, add_offset=False, batch_size=3)
f.close()
g.close()

예측 질병: 경계성 인격장애
증상 목록: 의심, 약물남용, 대인관계 어려움, 조절할 수 없는 충동감, 자살의 위험, 우울, 망상

시작 증상: 의심
asked: ['의심']
remain: []
new: []
asked: ['의심']
remain: ['말더듬', '다리 통증', '수면장애', '저림', '불안', '좁은 공간에서의 불안감']
new: ['말더듬', '다리 통증', '수면장애', '저림', '불안', '좁은 공간에서의 불안감']
asked: ['의심', '좁은 공간에서의 불안감', '다리 통증', '말더듬', '수면장애', '불안', '저림']
remain: ['말더듬', '다리 통증', '수면장애', '저림', '불안', '좁은 공간에서의 불안감', '말을 많이 함', '높아진 자존심', '우울', '망상', '사고의 비약', '긴장', '원한', '권리를 강하게 주장', '질투']
asked: ['의심', '좁은 공간에서의 불안감', '다리 통증', '말더듬', '수면장애', '불안', '저림', '긴장', '사고의 비약', '망상']
remain: ['수면장애', '식욕부진', '환각', '자살의 위험', '삶에 대한 흥미 상실', '오후 우울감 완화', '불안', '좁은 공간에서의 불안감', '감각 과민', '대인관계 어려움', '감정 변화', '체중증가', '복부 통증', '폭식', '구토', '불안', '반복적인 동작', '말더듬']
asked: ['의심', '좁은 공간에서의 불안감', '다리 통증', '말더듬', '수면장애', '불안', '저림', '긴장', '사고의 비약', '망상', '삶에 대한 흥미 상실', '대인관계 어려움']
remain: []
new: []
asked: ['의심', '좁은 공간에서의 불안감', '다리 통증', '말더듬', '수면장애', '불안', '저림', '긴장', '사고의 비약', '망상', '삶에 대한 흥미 상실', '대인관계 어려움']
remain: ['약물남용', '직무 태만', '나쁜 품행', '야

KeyboardInterrupt: ignored

In [ ]:
total_symps = 0
for (key, full) in zip(data_original['disease'], data_original['symptoms']):
    full_arr = []
    del_token = "[']"
    for t in del_token:
      full = full.replace(t, "")
    full = full.split(',')

    for t in full:
      if len(t) > 0 and t[0] == " ":
        t = t[1:]
      full_arr.append(t)

    for i in full_arr:
      total_symps += 1

total_symps